In [ ]:
# tfidf_lightgbm_colab.ipynb（または .py ファイルとしても使用可能）

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import joblib

# 1. データの読み込み
train_df = pd.read_csv("train.csv")

# 2. TF-IDFベクトル化（文字 n-gram）
vectorizer = TfidfVectorizer(
    analyzer='char',           # 文字単位で分析
    ngram_range=(3, 5),        # 3〜5グラム
    max_features=100000        # 最大特徴量数
)
X = vectorizer.fit_transform(train_df['URL'])
y = train_df['label']

# 3. モデル学習（3分割交差検証）
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
auc_scores = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
    print(f"\n📂 Fold {fold+1}")

    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    model = lgb.LGBMClassifier(
        n_estimators=100,         # 決定木の数
        max_depth=7,              # 最大深さ
        learning_rate=0.2,        # 学習率
        subsample=0.8,            # データのサブサンプリング
        colsample_bytree=0.8,     # 特徴量のサブサンプリング
        random_state=42,          # 再現性のための乱数シード
        n_jobs=-1                 # 並列処理
    )

    model.fit(X_train, y_train)
    val_pred = model.predict_proba(X_val)[:, 1]
    auc = roc_auc_score(y_val, val_pred)
    auc_scores.append(auc)
    print(f"✅ Fold {fold+1} ROC-AUC: {auc:.5f}")

print("\n📈 平均 ROC-AUC:", np.mean(auc_scores))

# 4. ベクトライザーとモデルの保存
joblib.dump(vectorizer, "tfidf_vectorizer.pkl")
joblib.dump(model, "final_model.pkl")
